In [1]:
import numpy as np
from openbb import obb
obb.user.preferences.output_type = "dataframe"
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from datetime import datetime, timedelta

# Strategy: Buy SPY if it had a big decline on Tuesday 
# hold for specified number of days

## Specify parameters for analysis

In [2]:
holding_period = 4
start_date = "2020-02-01"
end_date = pd.Timestamp.today().strftime("%Y-%m-%d")

## Get data

In [3]:
# Get daily price data
daily_ohlc = spy = obb.equity.price.historical(
    "SPY",
    start_date = start_date,
    end_date = end_date,
    provider="yfinance"
)
daily_ohlc.index = pd.to_datetime(daily_ohlc.index).tz_localize("US/Eastern")

In [4]:
daily_ohlc

,open,high,low,close,volume,dividend
date,,,,,,
2020-02-03 00:00:00-05:00,323.350006,326.160004,323.220001,324.119995,69083000,0.0
2020-02-04 00:00:00-05:00,328.070007,330.010010,327.720001,329.059998,62573200,0.0
2020-02-05 00:00:00-05:00,332.269989,333.089996,330.670013,332.859985,65951100,0.0
2020-02-06 00:00:00-05:00,333.910004,334.190002,332.799988,333.980011,50359700,0.0
2020-02-07 00:00:00-05:00,332.820007,333.989990,331.600006,332.200012,64139400,0.0
...,...,...,...,...,...,...
2025-02-10 00:00:00-05:00,604.030029,605.500000,602.739990,604.849976,26048700,0.0
2025-02-11 00:00:00-05:00,602.549988,605.859985,602.429993,605.309998,30056700,0.0
2025-02-12 00:00:00-05:00,599.200012,604.549988,598.510010,603.359985,45076100,0.0


## Identify trade setups

### Set up trade criteria

We want to evaluate Tuesdays on which SPY had a big loss, defined as the price declining by an amount greater than the 30-day average true range.

In [5]:
# 30-day average true range (ATR)
daily_ohlc['true_range'] = (np.maximum(daily_ohlc['high'], daily_ohlc['close'].shift(1))
                            - np.minimum(daily_ohlc['low'], daily_ohlc['close'].shift(1)))
daily_ohlc['atr'] = daily_ohlc['true_range'].rolling(30).mean()

In [6]:
# A buy setup is a Wednesday with a price decline greater than the ATR
daily_ohlc['buy_setup_int'] = 0
daily_ohlc.loc[((daily_ohlc.index.dayofweek == 1) 
                & (daily_ohlc['close'] - daily_ohlc['close'].shift(1) <= - daily_ohlc['atr'].shift(1))),
               'buy_setup_int'] = 1

In [7]:
daily_ohlc.iloc[-20:]

,open,high,low,close,volume,dividend,true_range,atr,buy_setup_int
date,,,,,,,,,
2025-01-17 00:00:00-05:00,596.960022,599.359985,595.609985,597.580017,58070600,0.0,7.719971,7.158667,0
2025-01-21 00:00:00-05:00,600.669983,603.059998,598.669983,603.049988,42532900,0.0,5.479980,7.208000,0
2025-01-22 00:00:00-05:00,605.919983,607.820007,605.359985,606.440002,48196000,0.0,4.770020,7.294334,0
2025-01-23 00:00:00-05:00,605.799988,609.750000,605.520020,609.750000,41152100,0.0,4.229980,7.354999,0
2025-01-24 00:00:00-05:00,609.809998,610.780029,606.799988,607.969971,34604700,0.0,3.980042,7.361668,0
2025-01-27 00:00:00-05:00,594.809998,599.690002,594.640015,599.369995,70361100,0.0,13.329956,7.683667,0
2025-01-28 00:00:00-05:00,600.619995,605.369995,597.250000,604.520020,44433300,0.0,8.119995,7.766667,0
2025-01-29 00:00:00-05:00,603.719971,604.130005,599.219971,601.809998,37177400,0.0,5.300049,7.839001,0
2025-01-30 00:00:00-05:00,603.960022,606.599976,600.719971,605.039978,39281300,0.0,5.880005,7.891001,0


### Set target position for each period

In [8]:
# Set target position to 1 (100%) for the holding period after a buy setup
daily_ohlc.loc[daily_ohlc['buy_setup_int'].rolling(holding_period).sum() >= 1, 'target_position'] = 1.00
daily_ohlc.loc[daily_ohlc['buy_setup_int'].rolling(holding_period).sum() < 1, 'target_position'] = 0.00

In [9]:
daily_ohlc.loc[daily_ohlc['target_position'] == 1]

,open,high,low,close,volume,dividend,true_range,atr,buy_setup_int,target_position
date,,,,,,,,,,
2020-09-08 00:00:00-04:00,336.709991,342.640015,332.880005,333.209991,114465300,0.0,9.760010,4.243336,1,1.0
2020-09-09 00:00:00-04:00,337.549988,342.459991,336.609985,339.790009,91462300,0.0,9.250000,4.458669,0,1.0
2020-09-10 00:00:00-04:00,341.820007,342.529999,332.850006,333.890015,90569500,0.0,9.679993,4.629336,0,1.0
2020-09-11 00:00:00-04:00,335.820007,336.970001,331.000000,334.059998,84680200,0.0,5.970001,4.645670,0,1.0
2021-05-11 00:00:00-04:00,413.100006,415.269989,410.059998,414.209991,116888000,0.0,7.880005,3.709669,1,1.0
...,...,...,...,...,...,...,...,...,...,...
2024-09-06 00:00:00-04:00,549.940002,551.599976,539.440002,540.359985,68493800,0.0,12.159973,8.696007,0,1.0
2025-01-07 00:00:00-05:00,597.419983,597.750000,586.780029,588.630005,60393100,0.0,10.969971,6.223995,1,1.0
2025-01-08 00:00:00-05:00,588.700012,590.580017,585.200012,589.489990,47304700,0.0,5.380005,6.303328,0,1.0


## Calculate net changes of entire time period (population) and of trades (sample)

In [10]:
# Select the days with buy or sell setups
transactions = daily_ohlc.loc[
    ((daily_ohlc['target_position'] == 1) & (daily_ohlc['target_position'].shift(1) == 0))
    | ((daily_ohlc['target_position'] == 0) & (daily_ohlc['target_position'].shift(1) == 1))][['close', 'buy_setup_int']]

In [11]:
# Calculate forward return
transactions['exit_close'] = transactions['close'].shift(-1)
transactions['y_log_return'] = np.log(transactions['close'].shift(-1)) - np.log(transactions['close'])

In [12]:
transactions

,close,buy_setup_int,exit_close,y_log_return
date,,,,
2020-09-08 00:00:00-04:00,333.209991,1,338.459991,0.015633
2020-09-14 00:00:00-04:00,338.459991,0,414.209991,0.201967
2021-05-11 00:00:00-04:00,414.209991,1,415.519989,0.003158
2021-05-17 00:00:00-04:00,415.519989,0,433.720001,0.042868
2021-09-28 00:00:00-04:00,433.720001,1,428.640015,-0.011782
2021-10-04 00:00:00-04:00,428.640015,0,455.559998,0.060910
2021-11-30 00:00:00-05:00,455.559998,1,458.790009,0.007065
2021-12-06 00:00:00-05:00,458.790009,0,456.489990,-0.005026
2022-01-18 00:00:00-05:00,456.489990,1,439.839996,-0.037156


In [13]:
trades = transactions.loc[transactions['buy_setup_int'] == 1].dropna()

In [14]:
trades

,close,buy_setup_int,exit_close,y_log_return
date,,,,
2020-09-08 00:00:00-04:00,333.209991,1,338.459991,0.015633
2021-05-11 00:00:00-04:00,414.209991,1,415.519989,0.003158
2021-09-28 00:00:00-04:00,433.720001,1,428.640015,-0.011782
2021-11-30 00:00:00-05:00,455.559998,1,458.790009,0.007065
2022-01-18 00:00:00-05:00,456.489990,1,439.839996,-0.037156
2022-04-26 00:00:00-04:00,416.100006,1,414.480011,-0.003901
2022-09-13 00:00:00-04:00,393.100006,1,388.549988,-0.011642
2023-02-21 00:00:00-05:00,399.089996,1,397.730011,-0.003414
2023-03-07 00:00:00-05:00,398.269989,1,385.359985,-0.032952


In [15]:
# Get log returns for the entire population based on the holding period
population = daily_ohlc.iloc[::holding_period][['close', 'buy_setup_int']]

In [16]:
# Calculate forward net return
population['exit_close'] = population['close'].shift(-1)
population['y_log_return'] = np.log(population['close'].shift(-1)) - np.log(population['close'])

In [17]:
population

,close,buy_setup_int,exit_close,y_log_return
date,,,,
2020-02-03 00:00:00-05:00,324.119995,0,332.200012,0.024623
2020-02-07 00:00:00-05:00,332.200012,0,337.059998,0.014524
2020-02-13 00:00:00-05:00,337.059998,0,336.950012,-0.000326
2020-02-20 00:00:00-05:00,336.950012,0,311.500000,-0.078535
2020-02-26 00:00:00-05:00,311.500000,0,300.239990,-0.036817
...,...,...,...,...
2025-01-21 00:00:00-05:00,603.049988,0,599.369995,-0.006121
2025-01-27 00:00:00-05:00,599.369995,0,601.820007,0.004079
2025-01-31 00:00:00-05:00,601.820007,0,606.320007,0.007450


## Calculate backtest statistics

In [18]:
trades['y_log_return'].describe()

count    20.000000
mean      0.000417
std       0.017285
min      -0.037156
25%      -0.010493
50%       0.003255
75%       0.009689
max       0.028651
Name: y_log_return, dtype: float64

In [19]:
print("The sample mean log return is ", trades['y_log_return'].mean())
print("The population mean log return is ", population['y_log_return'].mean())

The sample mean log return is  0.00041673530626544774
The population mean log return is  0.001966456350329893


In [20]:
print("Backtest sample statistics (log returns):")
print("Mean: ", trades['y_log_return'].mean())
print("Standard deviation: ", trades['y_log_return'].std())
print("N: ", trades['y_log_return'].count())
print("t: ", (trades['y_log_return'].mean() - population['y_log_return'].mean()) /
      (trades['y_log_return'].std() / (trades['y_log_return'].count() ** 0.5)))

Backtest sample statistics (log returns):
Mean:  0.00041673530626544774
Standard deviation:  0.01728489749872261
N:  20
t:  -0.4009606190544246


In [21]:
trades.iloc[-10:]

,close,buy_setup_int,exit_close,y_log_return
date,,,,
2023-05-02 00:00:00-04:00,410.839996,1,412.739990,0.004614
2023-05-23 00:00:00-04:00,414.089996,1,420.179993,0.014600
2023-08-15 00:00:00-04:00,442.890015,1,439.339996,-0.008048
2023-09-26 00:00:00-04:00,425.880005,1,427.309998,0.003352
2023-10-03 00:00:00-04:00,421.589996,1,432.290009,0.025063
2024-02-13 00:00:00-05:00,494.079987,1,496.760010,0.005410
2024-03-05 00:00:00-05:00,507.179993,1,511.279999,0.008051
2024-04-30 00:00:00-04:00,501.980011,1,516.570007,0.028651
2024-09-03 00:00:00-04:00,552.080017,1,546.409973,-0.010323
